Preprocessing (pipeline)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Load the data

In [4]:
data_train = pd.read_csv('../datasets/train.csv')
data_test = pd.read_csv('../datasets/test.csv')

#### Check the correlation

#### Decide what I want to do with each column and create a pipeline for this